In [ ]:
!pip install mlflow -q

In [ ]:
from enum import Enum

from tensorflow.keras.applications import MobileNetV2, ResNet50
from tensorflow.keras.applications.mobilenet_v2 import (
    preprocess_input as preprocess_input_mobilenet,
)
from tensorflow.keras.applications.resnet import (
    preprocess_input as preprocess_input_resnet,
)
from tensorflow.keras.optimizers import SGD, Adam, RMSprop


class OptimizerType(Enum):
    ADAM = Adam
    SGD = SGD
    RMSPROP = RMSprop


class CNNType(str, Enum):
    RESNET = ResNet50
    MOBILENETV2 = MobileNetV2


optimizer_map = {
    "adam": OptimizerType.ADAM,
    "sgd": OptimizerType.SGD,
    "rmsprop": OptimizerType.RMSPROP,
}
cnn_map = {"resnet50": CNNType.RESNET, "mobilenetv2": CNNType.MOBILENETV2}
preprocess_img_map = {
    "resnet50": preprocess_input_resnet,
    "mobilenetv2": preprocess_input_mobilenet,
}

# ------ HYPERPARAMETERS (Default Values) ------
BATCH_SIZE = 12
EPOCHS = 10
LEARNING_RATE = 0.001
OPTIMIZER_str = "adam"
CNN_str = "resnet50"
FINETUNE_DEPTH = 100

# ------ RUN_ID ------
PREFECT_RUN_ID = "Default"
EXPERIMENT_NAME = "Default Experiment"
RUN_NAME = "Avocado_Ripening_Model"
MLFLOW_URI = "https://mlflow.lepcodes.com"
MODEL_PATH = "/kaggle/working/feature_extraction_model.keras"
MODEL_PATH_FT = "/kaggle/working/fine_tuned_model.keras"

In [ ]:
import os

import pandas as pd
import requests
import tensorflow as tf
from tensorflow.keras import Sequential

import mlflow

In [ ]:
try:
    response = requests.get(f"{MLFLOW_URI}/health")
    if response.status_code != 200 and response.status_code != 404:
        print(
            f"Server responded with code {response.status_code}, but seems to be alive."
        )
    else:
        print("Connection stablished with MLflow server.")

    mlflow.set_tracking_uri(MLFLOW_URI)
    print(f"Tracking URI set: {mlflow.get_tracking_uri()}")

except Exception as e:
    raise Exception(f"CRITICAL ERROR: Cannot connect to MLFlow. Cause: {e}")

In [ ]:
try:
    mlflow.set_experiment(EXPERIMENT_NAME)
    print(f'Experiment "{EXPERIMENT_NAME}" set!')
except:
    try:
        mlflow.create_experiment(
            name=EXPERIMENT_NAME, artifact_store="mlflow-artifacts:/"
        )
        mlflow.set_experiment(EXPERIMENT_NAME)
        print(f"{EXPERIMENT_NAME} don't existed. Created!")
    except Exception:
        raise Exception("Error configuring experiment")

In [ ]:
with mlflow.start_run(run_name=RUN_NAME + "_Head_Only") as run:
    print(f"Run {RUN_NAME} starting with ID: {run.info.run_id}")
    print(f"Prefect Run ID: {PREFECT_RUN_ID}")
    mlflow.set_tag("prefect_run_id", PREFECT_RUN_ID)

    print("Loading metrics...")
    mlflow.log_metric("val_loss", 0.1)
    
    print("Loading artifacts...")
    model = Sequential([
        tf.keras.layers.Lambda(lambda x: x)   # identity
    ])
    mlflow.tensorflow.log_model(model, "avocado-model")

In [ ]:
with mlflow.start_run(run_name=RUN_NAME + "_Fine_Tuned") as run:

    mlflow.set_tag("prefect_run_id", PREFECT_RUN_ID)
    
    print("Loading metrics...")
    mlflow.log_metric("val_loss", 0.1)
    
    print("Loading artifacts...")
    model = Sequential([
        tf.keras.layers.Lambda(lambda x: x)   # identity
    ])
    mlflow.tensorflow.log_model(model, "avocado-model")